In [12]:
from mxnet import gluon
from mxnet.gluon import nn
from mxnet import nd

net = gluon.nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(256, activation="relu"))
    net.add(nn.Dense(10))
print net

Sequential(
  (0): Dense(None -> 256, Activation(relu))
  (1): Dense(None -> 10, linear)
)


In [6]:
class MLP(nn.Block):
    def __init__(self, **kwargs):
        super(MLP, self).__init__(**kwargs)
        with self.name_scope():
            self.dense0 = nn.Dense(256)
            self.dense1 = nn.Dense(10)

    def forward(self, x):
        return self.dense1(nd.relu(self.dense0(x)))

In [17]:
class RecMLP(nn.Block):
    def __init__(self, **kwargs):
        super(RecMLP, self).__init__(**kwargs)
        self.net = nn.Sequential()
        with self.name_scope():
            self.net.add(nn.Dense(256, activation="relu"))
            self.net.add(nn.Dense(128, activation="relu"))
            self.dense0 = nn.Dense(256)
            self.dense1 = nn.Dense(128)
            self.dense2 = nn.Dense(64)
            self.denses = [self.dense0, self.dense1, self.dense2]
            # self.denses = [nn.Dense(256), nn.Dense(128), nn.Dense(64)] 
    
    def forward(self, x):
        x = self.net(x)
        for layer in self.denses:
            x = nd.relu(layer(x))
        return x

rec_mlp = nn.Sequential()
rec_mlp.add(MLP())
rec_mlp.add(nn.Dense(10))
print(rec_mlp)

Sequential(
  (0): MLP(
    (dense1): Dense(None -> 10, linear)
    (dense0): Dense(None -> 256, linear)
  )
  (1): Dense(None -> 10, linear)
)


In [8]:
nn.Dense

mxnet.gluon.nn.basic_layers.Dense

In [13]:
class Sequential(nn.Block):
    def __init__(self, **kwargs):
        super(Sequential, self).__init__(**kwargs)
    def add(self, block):
        self._children.append(block)
    def forward(self, x):
        for block in self._children:
            x = block(x)
        return x
net4 = Sequential()
with net4.name_scope():
    net4.add(nn.Dense(256, activation="relu"))
    net4.add(nn.Dense(10))

x = nd.random.uniform(shape=(4,20))
net4.initialize()
y = net4(x)
y


[[ 0.03126615  0.04562764  0.00039858 -0.08772382 -0.05355632  0.02904576
   0.08102556 -0.01433946 -0.04224151  0.06047883]
 [ 0.028719    0.03652266  0.00630052 -0.05650968 -0.07189324  0.08615956
   0.05951558 -0.06045965 -0.0299026   0.05651002]
 [ 0.02147349  0.04818896  0.05321142 -0.12616859 -0.06850231  0.09096346
   0.04064303 -0.05064794 -0.02200241  0.04859561]
 [ 0.03780478  0.07512389  0.03290458 -0.11641113 -0.03254965  0.05865289
   0.02542158 -0.01697343 -0.00049652  0.05892841]]
<NDArray 4x10 @cpu(0)>

In [18]:
rec_mlp = RecMLP()
rec_mlp.initialize()
print rec_mlp

RecMLP(
  (dense1): Dense(None -> 128, linear)
  (dense0): Dense(None -> 256, linear)
  (dense2): Dense(None -> 64, linear)
  (net): Sequential(
    (0): Dense(None -> 256, Activation(relu))
    (1): Dense(None -> 128, Activation(relu))
  )
)


In [19]:
def get_net():
    net = nn.Sequential()
    with net.name_scope():
        net.add(nn.Dense(4, activation="relu"))
        net.add(nn.Dense(2))
    return net

x = nd.random.uniform(shape=(3,5))

In [21]:
g_net = get_net()
print g_net
g_net.initialize()
g_net(x)

Sequential(
  (0): Dense(None -> 4, Activation(relu))
  (1): Dense(None -> 2, linear)
)



[[ 0.00223847  0.00497976]
 [ 0.00098496  0.00593723]
 [ 0.00051086 -0.00054389]]
<NDArray 3x2 @cpu(0)>

In [24]:
print g_net[0].weight, "\n", g_net[1].weight
print g_net[0].bias, "\n", g_net[1].bias

Parameter sequential14_dense0_weight (shape=(4L, 5L), dtype=<type 'numpy.float32'>) 
Parameter sequential14_dense1_weight (shape=(2L, 4L), dtype=<type 'numpy.float32'>)
Parameter sequential14_dense0_bias (shape=(4L,), dtype=<type 'numpy.float32'>) 
Parameter sequential14_dense1_bias (shape=(2L,), dtype=<type 'numpy.float32'>)


In [32]:
params = g_net.collect_params()
print params, type(params) , "\n"
pp = g_net.params
print pp, type(pp)

sequential14_ (
  Parameter sequential14_dense0_weight (shape=(4L, 5L), dtype=<type 'numpy.float32'>)
  Parameter sequential14_dense0_bias (shape=(4L,), dtype=<type 'numpy.float32'>)
  Parameter sequential14_dense1_weight (shape=(2L, 4L), dtype=<type 'numpy.float32'>)
  Parameter sequential14_dense1_bias (shape=(2L,), dtype=<type 'numpy.float32'>)
) <class 'mxnet.gluon.parameter.ParameterDict'> 

sequential14_ (

) <class 'mxnet.gluon.parameter.ParameterDict'>


In [28]:
from mxnet import init
params.initialize(init=init.Normal(sigma=0.02), force_reinit=True)
print(g_net[0].weight.data(), g_net[0].bias.data())

(
[[  3.47158033e-03   8.45357310e-03   6.62524346e-03  -3.85387614e-02
   -2.10823584e-02]
 [ -4.29782569e-02  -1.17376121e-02  -1.17822655e-03   2.53050011e-02
    1.06135905e-02]
 [ -4.21083048e-02  -3.17697268e-04  -1.17256511e-02  -1.05709750e-02
   -4.01610462e-03]
 [  1.69480536e-02   1.64358541e-02   1.71423808e-03   9.30319168e-03
    1.02952818e-05]]
<NDArray 4x5 @cpu(0)>, 
[ 0.  0.  0.  0.]
<NDArray 4 @cpu(0)>)


In [36]:
filename = "mlp.params"
g_net.save_params(filename)
print g_net(x)


[[ 0.00114251 -0.00036724]
 [ 0.00094571 -0.00030398]
 [ 0.00082506 -0.0002652 ]]
<NDArray 3x2 @cpu(0)>


In [37]:
import mxnet as mx
net2 = get_net()
net2.load_params(filename, mx.cpu())  # FIXME, gluon will support default ctx later
print(net2(x))


[[ 0.00114251 -0.00036724]
 [ 0.00094571 -0.00030398]
 [ 0.00082506 -0.0002652 ]]
<NDArray 3x2 @cpu(0)>


# 设计自定义的层

In [38]:
class CenteredLayer(nn.Block):
    def __init__(self, **kwargs):
        super(CenteredLayer, self).__init__(**kwargs)
        
    def forward(self, x):
        return x - x.mean()

In [39]:
layer = CenteredLayer()
layer(nd.array([1, 2, 3, 4, 5]))


[-2. -1.  0.  1.  2.]
<NDArray 5 @cpu(0)>

In [40]:
net = nn.Sequential()
with net.name_scope():
    net.add(nn.Dense(128))
    net.add(nn.Dense(10))
    net.add(CenteredLayer())

In [41]:
net.initialize()
y = net(nd.random_uniform(shape=(4, 8)))
y.mean()


[ -3.72529035e-10]
<NDArray 1 @cpu(0)>

In [42]:
my_param = gluon.Parameter("exciting_parameter_yay", shape=(3,3))
my_param.initialize()
my_param.data(), my_param.grad()

(
 [[ 0.01490656 -0.02265774 -0.0394236 ]
  [ 0.06461982 -0.00627637 -0.03756177]
  [ 0.05304176  0.06290463 -0.00108245]]
 <NDArray 3x3 @cpu(0)>, 
 [[ 0.  0.  0.]
  [ 0.  0.  0.]
  [ 0.  0.  0.]]
 <NDArray 3x3 @cpu(0)>)

In [45]:
class MyDense(nn.Block):
    def __init__(self, units, in_units, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        with self.name_scope():
            self.weight = self.params.get('weight', shape=(in_units, units))
            self.bias = self.params.get('bias', shape=(units,))
    
    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)


In [46]:
dense = MyDense(5, in_units=10, prefix='o_my_dense_')
dense.params

o_my_dense_ (
  Parameter o_my_dense_weight (shape=(10, 5), dtype=<type 'numpy.float32'>)
  Parameter o_my_dense_bias (shape=(5,), dtype=<type 'numpy.float32'>)
)

In [47]:
dense.initialize()
dense(nd.random.uniform(shape=(2, 10)))


[[ 0.0886663   0.06919234  0.09616455  0.06147449  0.        ]
 [ 0.06507732  0.          0.07136965  0.          0.        ]]
<NDArray 2x5 @cpu(0)>

In [49]:
nn.Dense

mxnet.gluon.nn.basic_layers.Dense

In [50]:
nd.array([1, 2, 3])


[ 1.  2.  3.]
<NDArray 3 @cpu(0)>